In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from random import seed, shuffle
import os,sys
import traceback
import numpy as np
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
from cvxpy import *
import cvxpy as cp
import numpy as np
import time


In [71]:
!pip3 install --upgrade pip
!pip install setuptools==57.5.0 #adde
!pip3 install dccp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.5.0
    Uninstalling setuptools-57.5.0:
      Successfully uninstalled setuptools-57.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


MessageError: ignored

In [137]:
# Load data
df = pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [138]:
import pandas as pd
import numpy as np
import cvxpy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Data Preprocessing
df['day_stayed'] = (pd.to_datetime(df["c_jail_out"]) - pd.to_datetime(df["c_jail_in"])).astype('timedelta64[h]') / 24
df = df[df.day_stayed > 0]

# Select relevant variables
df = df[['sex', 'age', 'age_cat', 'race', 'decile_score', 'score_text', 'priors_count', 'days_b_screening_arrest',
         'v_decile_score', 'v_score_text', 'c_charge_degree', 'is_violent_recid', 'is_recid', 'day_stayed', 'two_year_recid']]

# Filter for two races
df = df[df.race.isin(['African-American', 'Caucasian'])]

# Encoding categorical variables
df['sex'].replace(['Male', 'Female'], [1, 0], inplace=True)
df['race'].replace(['Caucasian', 'African-American'], [1, 0], inplace=True)
df['score_text'].replace(['Low', 'Medium', 'High'], [0, 1, 2], inplace=True)
df['v_score_text'].replace(['Low', 'Medium', 'High'], [0, 1, 2], inplace=True)
df['age_cat'].replace(['Less than 25', '25 - 45', 'Greater than 45'], [0, 1, 2], inplace=True)
df['c_charge_degree'].replace(['M', 'F'], [1, 0], inplace=True)

# Feature matrix and labels
X = df.drop('two_year_recid', axis=1).values
intercept = np.ones((X.shape[0], 1))  # Create a column of ones
X = np.hstack((intercept, X))
y = df['two_year_recid'].values
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}
sensitive_attr = df['race'].values
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test, sensitive_attr_train, sensitive_attr_test = train_test_split(
    X, y, sensitive_attr, test_size=0.2, random_state=42)

# Fairness Constraint Function
def fairness_constraint(X, y, sensitive_attr, theta, c):
    yX = cp.multiply(y, X @ theta)
    group0 = sensitive_attr == 0
    group1 = sensitive_attr == 1

    N0 = cp.sum(group0)
    N1 = cp.sum(group1)

    avg_group0 = cp.sum(cp.pos(yX[group0])) / N0
    avg_group1 = cp.sum(cp.pos(yX[group1])) / N1

    return [avg_group0 <= avg_group1 + c, avg_group0 >= avg_group1 - c]

# Training Function with Fairness Constraints
def train_fair_logistic_regression(X, y, sensitive_attr, c=0, solver=None):
    num_features = X.shape[1]
    theta = cp.Variable((num_features, 1))

    if y.ndim == 1:
        y = y[:, np.newaxis]

    logistic_loss = cp.sum(cp.logistic(-cp.multiply(y, X @ theta)))
    constraints = fairness_constraint(X, y, sensitive_attr, theta, c)

    problem = cp.Problem(cp.Minimize(logistic_loss), constraints)
    problem.solve(solver=solver)

    if problem.status not in ["infeasible", "unbounded"]:
        theta_value = theta.value.flatten()
        print("Problem solved successfully.")
        return theta_value
    else:
        print(f"Problem status: {problem.status}.")
        return None

# Prediction Function
def predict(X, theta):
    probs = 1 / (1 + np.exp(-X @ theta))
    return (probs >= 0.5).astype(int)

# Training the Model
c_tolerance = 0.01
theta_hat = train_fair_logistic_regression(X_train, y_train, sensitive_attr_train, c=c_tolerance, solver=cp.ECOS)

# Evaluating the Model
if theta_hat is not None:
    y_pred = predict(X_test, theta_hat)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.4f}")
else:
    print("Model training was unsuccessful")


DCPError: ignored

In [139]:
print("Initial shape of X:", X.shape)

# If X initially has 14 features, add an intercept term
if X.shape[1] == 14:
    intercept = np.ones((X.shape[0], 1))  # Column of ones
    X = np.hstack((intercept, X))  # Add the intercept column to the feature matrix

# Check the shape after adding the intercept
print("Shape of X after adding intercept:", X.shape)

Initial shape of X: (5699, 15)
Shape of X after adding intercept: (5699, 15)


In [127]:
intercept = np.ones((X.shape[0], 1))  # Column of ones
X = np.hstack((intercept, X))  # Add the intercept column to the feature matrix
X.shape

(5699, 16)

In [35]:
# Calculate ce(['Male','Female'], [1,0], inplace = True)
df['race'].replace(['Caucasian','African-American'], [1,0], inplace = True)
df['score_text'].replace(['Low','Medium','High'], [0,1,2], inplace = True)
df['v_score_text'].replace(['Low','Medium','High'], [0,1,2], inplace = True)
df['age_cadays in jail for each individual
df['day_stayed'] = (pd.to_datetime(df["c_jail_out"]) - pd.to_datetime(df["c_jail_in"])).astype('timedelta64[h]') / 24
# Remove those who were in jail for negative days
df = df[df.day_stayed > 0]
# Variable selection
# Drop unnecessary variables (eg.: id, name, first, etc.)
# Drop variables that have repeated info (eg.: dob to general and related to age, age_cat)
df = df[['sex', 'age', 'age_cat', 'race', 'decile_score', 'score_text', 'priors_count',
             'days_b_screening_arrest', 'v_decile_score', 'v_score_text','c_charge_degree',
             'is_violent_recid', 'is_recid', 'day_stayed', 'two_year_recid']]
# df.race.value_counts() -> Not all are African American or Caucasian
# Drop those that are not
is_race = [i=='African-American' or i=='Caucasian' for i in df.race]
df = df.loc[is_race]
df.race.value_counts()
df['sex'].replat'].replace(['Less than 25','25 - 45','Greater than 45'], [0,1,2],inplace = True)
df['c_charge_degree'].replace(['M','F'], [1,0], inplace = True)
df.head(5)

,sex,age,age_cat,race,decile_score,score_text,priors_count,days_b_screening_arrest,v_decile_score,v_score_text,c_charge_degree,is_violent_recid,is_recid,day_stayed,two_year_recid
1,1,34,1,0,3,0,0,-1.0,1,0,0,1,1,10.041667,1
2,1,24,0,0,4,0,4,-1.0,3,0,0,0,1,1.083333,1
6,1,41,1,1,6,1,14,-1.0,2,0,0,0,1,6.291667,1
8,0,39,1,1,1,0,0,-1.0,1,0,1,0,0,2.916667,0
9,1,21,0,1,3,0,1,428.0,5,1,0,1,1,0.958333,1


,sex,age,age_cat,race,decile_score,score_text,priors_count,days_b_screening_arrest,v_decile_score,v_score_text,c_charge_degree,is_violent_recid,is_recid,day_stayed,two_year_recid
0,Male,69,Greater than 45,Other,1,Low,0,-1.0,1,Low,F,0,0,0.958333,0
1,Male,34,25 - 45,African-American,3,Low,0,-1.0,1,Low,F,1,1,10.041667,1
2,Male,24,Less than 25,African-American,4,Low,4,-1.0,3,Low,F,0,1,1.083333,1
5,Male,44,25 - 45,Other,1,Low,0,0.0,1,Low,M,0,0,1.291667,0
6,Male,41,25 - 45,Caucasian,6,Medium,14,-1.0,2,Low,F,0,1,6.291667,1


In [76]:
X = df.drop('two_year_recid', axis=1).values
y = df['two_year_recid'].values
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}

# Sensitive attribute (assuming 'race' is the sensitive attribute)
sensitive_attr = df['race'].values

In [77]:
X_train,X_test,y_train,y_test,x_race_train,x_race_test = train_test_split(X,y,sensitive_attr,test_size=1/7,random_state=5243)
print(X_train.shape,X_test.shape,X.shape)

(4884, 14) (815, 14) (5699, 14)


In [81]:
X.shape, y.shape

((5699, 14), (5699,))

In [79]:
import numpy as np
from scipy.optimize import minimize
# Split the dataset
X = df.drop('two_year_recid', axis=1).values
y = df['two_year_recid'].values
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}

# Sensitive attribute (assuming 'race' is the sensitive attribute)
sensitive_attr = df['race'].values

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test, sensitive_attr_train, sensitive_attr_test = train_test_split(
    X, y, sensitive_attr, test_size=1/7, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

class FairLogisticRegression:
    def __init__(self, c=0.01):
        self.c = c  # Fairness tolerance parameter

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y, sensitive_attr):
        # Verify dimensions of X and reshape theta
        num_features = X.shape[1]
        theta_init = np.zeros(num_features)

        # Custom loss function
        def custom_loss(theta):
            y_pred = self.sigmoid(X @ theta)
            loss = -np.sum(np.log(y_pred[y == 1])) - np.sum(np.log(1 - y_pred[y == -1]))

            # Add fairness constraint
            N0 = np.sum(sensitive_attr == 0)
            N1 = np.sum(sensitive_attr == 1)
            fairness_loss = (
                np.sum(np.maximum(0, -y_pred[sensitive_attr == 0])) / N0 +
                np.sum(np.maximum(0, y_pred[sensitive_attr == 1])) / N1
            ) * self.c

            return loss + fairness_loss

        # Use optimization to find the best theta
        result = minimize(
            custom_loss,
            theta_init,
            method="L-BFGS-B"
        )

        self.theta_ = result.x

    def predict(self, X):
        return np.sign(X @ self.theta_)

# Example usage
# X_train: Training features
# y_train: Training labels (should be -1 or 1)
# sensitive_attr_train: Sensitive attribute for training data (binary)

# Train the FairLogisticRegression model
fair_logreg = FairLogisticRegression(c=0.01)
fair_logreg.fit(X_train, y_train, sensitive_attr_train)

# Make predictions on the test set
y_pred = fair_logreg.predict(X_test)


In [82]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Function to calculate fairness metrics
def calculate_fairness_metrics(y_true, y_pred, sensitive_attr):
    # Calculating confusion matrix for each group
    cm_0 = confusion_matrix(y_true[sensitive_attr == 0], y_pred[sensitive_attr == 0])
    cm_1 = confusion_matrix(y_true[sensitive_attr == 1], y_pred[sensitive_attr == 1])

    # Extracting true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN)
    TP_0, FP_0, FN_0, TN_0 = cm_0.ravel()
    TP_1, FP_1, FN_1, TN_1 = cm_1.ravel()

    # Calculating False Positive Rate (FPR) and False Negative Rate (FNR)
    FPR_0 = FP_0 / (FP_0 + TN_0)
    FPR_1 = FP_1 / (FP_1 + TN_1)
    FNR_0 = FN_0 / (FN_0 + TP_0)
    FNR_1 = FN_1 / (FN_1 + TP_1)

    # Disparate impact in terms of FPR and FNR
    DFPR = FPR_0 - FPR_1
    DFNR = FNR_0 - FNR_1

    return DFPR, DFNR

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
DFPR, DFNR = calculate_fairness_metrics(y_test, y_pred, sensitive_attr_test)

print(f"Accuracy: {accuracy}")
print(f"Disparate False Positive Rate (DFPR): {DFPR}")
print(f"Disparate False Negative Rate (DFNR): {DFNR}")

# You can also compare these metrics with those of an unconstrained model or other baseline models


Accuracy: 0.9742331288343559
Disparate False Positive Rate (DFPR): 0.02680675569520817
Disparate False Negative Rate (DFNR): 0.0


In [112]:
import numpy as np
import cvxpy as cp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming 'df' is your preprocessed DataFrame
# Preprocess your data as you did before
# ...

# Split the dataset
X = df.drop('two_year_recid', axis=1).values
y = df['two_year_recid'].values
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}

# Sensitive attribute (assuming 'race' is the sensitive attribute)
sensitive_attr = df['race'].values

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test, sensitive_attr_train, sensitive_attr_test = train_test_split(
    X, y, sensitive_attr, test_size=1/7, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

import numpy as np
import cvxpy as cp
# Reviewing and correcting the 'fairness_constraint' function

def fairness_constraint(X, y, sensitive_attr, theta, c):
    """
    Construct fairness constraints for logistic regression.
    X: Feature matrix
    y: Labels
    sensitive_attr: Sensitive attribute array (binary)
    theta: Logistic regression coefficients (CVXPY variable)
    c: Tolerance level for fairness
    """
    yX = cp.multiply(y, X @ theta)
    group0 = sensitive_attr == 0
    group1 = sensitive_attr == 1

    N0 = cp.sum(group0)
    N1 = cp.sum(group1)

    avg_group0 = cp.sum(cp.pos(yX[group0])) / N0
    avg_group1 = cp.sum(cp.pos(yX[group1])) / N1

    return [avg_group0 <= avg_group1 + c, avg_group0 >= avg_group1 - c]


In [113]:
def train_fair_logistic_regression(X, y, sensitive_attr, c=0, solver=None):
    num_features = X.shape[1]  # Number of features in X
    theta = cp.Variable((num_features, 1))  # theta should match the number of features
    X = np.hstack([np.ones((X.shape[0], 1)), X])

    # Ensure y is a column vector
    if y.ndim == 1:
        y = y[:, np.newaxis]

    logistic_loss = cp.sum(cp.logistic(-cp.multiply(y, X @ theta)))

    # Fairness constraints
    constraints = fairness_constraint(X, y, sensitive_attr, theta, c)

    # Solve the problem
    problem = cp.Problem(cp.Minimize(logistic_loss), constraints)
    problem.solve(solver=solver)

    if problem.status not in ["infeasible", "unbounded"]:
        # Reshape theta to be a 1D array for compatibility with other operations
        theta_value = theta.value.flatten()
        print("Problem solved successfully.")
        print("Optimal value of theta:", theta_value)
        return theta_value
    else:
        # Handle the infeasible or unbounded case
        print(f"Problem status: {problem.status}.")
        print("The problem is either infeasible or unbounded.")
        return None


In [114]:
def predict(X, theta):
    """
    Make predictions using the logistic regression model.
    X: Feature matrix
    theta: Model parameters
    """
    probs = 1 / (1 + np.exp(-X @ theta))
    return (probs >= 0.5).astype(int)


In [115]:
from sklearn.metrics import accuracy_score

# Example usage
c_tolerance = 0.01  # Adjust the tolerance
theta_hat = train_fair_logistic_regression(X_train, y_train, sensitive_attr_train, c=c_tolerance, solver=cp.ECOS)

if theta_hat is not None:
    # Making predictions on the test set
    y_pred = predict(X_test, theta_hat)
    y_pred = np.where(y_pred == -1, 0, 1)  # Convert to 0 and 1 if needed
    y_test_converted = np.where(y_test == -1, 0, 1)  # Convert to 0 and 1 if needed

    # Calculate accuracy
    accuracy = accuracy_score(y_test_converted, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
else:
    print("Model training was unsuccessful. Please check the constraints and data.")


ValueError: ignored

In [109]:
logistic_loss = cp.sum(cp.logistic(-cp.multiply(y, X @ theta)))

# Check if the loss function is DCP compliant
print("Logistic loss is DCP:", logistic_loss.is_dcp())

# Check each constraint
for constraint in constraints:
    print("Constraint is DCP:", constraint.is_dcp())

ValueError: ignored

In [99]:
def train_fair_logistic_regression(X, y, sensitive_attr, c=0, solver=None):
    num_features = X.shape[1]
    theta = cp.Variable((num_features, 1))  # Ensure theta is a column vector

    # Rest of your code...

    if problem.status not in ["infeasible", "unbounded"]:
        # Reshape theta to be a 1D array for compatibility with other operations
        theta_value = theta.value.flatten()
        print("Problem solved successfully.")
        print("Optimal value of theta:", theta_value)
        return theta_value
    else:
        # Handle the infeasible or unbounded case
        print(f"Problem status: {problem.status}.")
        print("The problem is either infeasible or unbounded.")
        return None


In [86]:
from sklearn.metrics import accuracy_score

def predict(X, theta):
    """
    Make predictions using the logistic regression model.
    X: Feature matrix
    theta: Model parameters
    """
    # Compute the probabilities using the logistic function
    probs = 1 / (1 + np.exp(-X @ theta))
    # Convert probabilities to binary predictions (0 or 1)
    return (probs >= 0.5).astype(int)

# Making predictions on the test set
y_pred = predict(X_test, theta_hat)

# Convert predictions and true labels to the same format (0 or 1)
y_pred = np.where(y_pred == -1, 0, 1)
y_test_converted = np.where(y_test == -1, 0, 1)

# Calculate accuracy
accuracy = accuracy_score(y_test_converted, y_pred)
print(f"Accuracy: {accuracy:.4f}")


ValueError: ignored

In [87]:
# Check the shape of theta_hat
print("Shape of theta_hat:", theta_hat.shape)

# If theta_hat is not a 1D array or is empty, there might be an issue
if theta_hat.ndim != 1 or theta_hat.size == 0:
    print("Error: theta_hat is not in the correct shape or is empty. Please check the training phase.")
else:
    # Continue with making predictions
    y_pred = predict(X_test, theta_hat)

    # Convert predictions and true labels to the same format (0 or 1)
    y_pred = np.where(y_pred == -1, 0, 1)
    y_test_converted = np.where(y_test == -1, 0, 1)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_converted, y_pred)
    print(f"Accuracy: {accuracy:.4f}")


AttributeError: ignored

In [66]:
from __future__ import division
import os,sys
import numpy as np
import traceback

sys.path.insert(0, "/home/mzafar/libraries/dccp") # we will store the latest version of DCCP here.
from cvxpy import *
import dccp
from dccp.problem import is_dccp






class LinearClf():


    def __init__(self, loss_function, lam=None, train_multiple=False, random_state=1234):

        """
            Model can be logistic regression or linear SVM in primal form

            We will define the lam parameter once and for all for a single object.
            For cross validating multiple models, we will write a function for doing that.

        """


        """ Setting default lam val and Making sure that lam is provided for each group """
        if lam is None:
            if train_multiple == False:
                lam = 0.0
            else:
                lam = {0:0.0, 1:0.0}

        else:
            if train_multiple == True:
                assert(isinstance(lam, dict))


        self.loss_function = loss_function
        self.lam = lam
        self.train_multiple = train_multiple


        np.random.seed(random_state)


    def fit(self, X, y, x_sensitive, cons_params=None):

        """
            X: n x d array
            y: n length vector
            x_sensitive: n length vector


            cons_params will be a dictionary
            cons_params["tau"], cons_params["mu"] and cons_params["EPS"] are the solver related parameters. Check DCCP documentation for details
            cons_params["cons_type"] specified which type of constraint to apply
                - cons_type = -1: No constraint
                - cons_type = 0: Parity
                - cons_type = 1: Preferred impact
                - cons_type = 2: Preferred treatment
                - cons_type = 3: Preferred both

            cons_params["s_val_to_cons_sum"]: The ramp approximation -- only needed for cons_type 1 and 3
        """


        """
            Setting up the initial variables
        """

        max_iters = 100 # for CVXPY convex solver
        max_iter_dccp = 50  # for the dccp. notice that DCCP hauristic runs the convex program iteratively until arriving at the solution




        """
            Construct the optimization variables
        """

        constraints = []

        np.random.seed(1234) # set the seed before initializing the values of w
        if self.train_multiple == True:
            w = {}
            for k in set(x_sensitive):
                w[k] = Variable(X.shape[1]) # this is the weight vector
                w[k].value = np.random.rand(X.shape[1]) # initialize the value of w -- uniform distribution over [0,1]
        else:
            w = Variable(X.shape[1]) # this is the weight vector
            w.value = np.random.rand(X.shape[1])



        """
            Solve the optimization problem here
        """

        num_all = X.shape[0] # set of all data points

        if self.train_multiple == True:

            obj = 0
            for k in set(x_sensitive):
                idx = x_sensitive==k
                X_k = X[idx]
                y_k = y[idx]
                obj += sum_squares(w[k][1:]) * self.lam[k] # first term in w is the intercept, so no need to regularize that

                if self.loss_function == "logreg":
                    obj += sum_entries(  logistic( mul_elemwise(-y_k, X_k*w[k]) )  ) / num_all # notice that we are dividing by the length of the whole dataset, and not just of this sensitive group. this way, the group that has more people contributes more to the loss

                elif self.loss_function == "svm_linear":
                    obj += sum_entries ( max_elemwise (0, 1 - mul_elemwise ( y_k,  X_k*w[k])) ) / num_all

                else:
                    raise Exception("Invalid loss function")

        else:

            obj = 0
            obj += sum_squares(w[1:]) * self.lam # regularizer -- first term in w is the intercept, so no need to regularize that
            if self.loss_function == "logreg":
                obj += sum_entries(  logistic( mul_elemwise(-y, X*w) )  ) / num_all
            elif self.loss_function == "svm_linear":
                obj += sum_entries ( max_elemwise (0, 1 - mul_elemwise ( y,  X*w)) ) / num_all
            else:
                raise Exception("Invalid loss function")


        """
            Constraints here
        """
        if cons_params is not None:

            cons_type = cons_params["cons_type"]
            if cons_type == -1: # no constraint
                pass
            elif cons_type == 0: # disp imp with single boundary
                cov_thresh = np.abs(0.) # perfect fairness -- see our AISTATS paper for details
                constraints += self.get_di_cons_single_boundary(X, y, x_sensitive, w, cov_thresh)
            elif cons_type in [1,3]: # preferred imp, pref imp + pref treat
                constraints += self.get_preferred_cons(X, x_sensitive, w, cons_type, cons_params["s_val_to_cons_sum"])
            elif cons_type == 2:
                constraints += self.get_preferred_cons(X, x_sensitive, w, cons_type)
            else:
                raise Exception("Wrong constraint type")



        prob = Problem(Minimize(obj), constraints)
        # print "Problem is DCP (disciplined convex program):", prob.is_dcp()
        # print "Problem is DCCP (disciplined convex-concave program):", is_dccp(prob)


        """
            Solving the problem
        """

        try:

            tau, mu, EPS = 0.5, 1.2, 1e-4 # default dccp parameters, need to be varied per dataset
            if cons_params is not None: # in case we passed these parameters as a part of dccp constraints
                if cons_params.get("tau") is not None: tau = cons_params["tau"]
                if cons_params.get("mu") is not None: mu = cons_params["mu"]
                if cons_params.get("EPS") is not None: EPS = cons_params["EPS"]

            prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                verbose=False,
                feastol=EPS, abstol=EPS, reltol=EPS,feastol_inacc=EPS, abstol_inacc=EPS, reltol_inacc=EPS,
                max_iters=max_iters, max_iter=max_iter_dccp)


            # print "Optimization done, problem status:", prob.status
            assert(prob.status == "Converged" or prob.status == "optimal")


            # check that the fairness constraint is satisfied
            for f_c in constraints:
                try:
                    assert(f_c.value == True)
                except:
                    print "Assertion failed. Fairness constraints not satisfied."
                    print traceback.print_exc()
                    sys.stdout.flush()
                    return
                    # sys.exit(1)

        except:
            traceback.print_exc()
            sys.stdout.flush()
            # sys.exit(1)
            return


        """
            Storing the results
        """

        if self.train_multiple == True:
            self.w = {}
            for k in set(x_sensitive):
                self.w[k] = np.array(w[k].value).flatten() # flatten converts it to a 1d array
        else:
            self.w = np.array(w.value).flatten() # flatten converts it to a 1d array


        return self.w


    def decision_function(self, X, k=None):
        """ Predicts labels for all samples in X

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.
        Returns

        k: the group whose decision boundary should be used.
        k = None means that we trained one clf for the whole dataset
        -------
        y : array of shape = [n_samples]
        """

        if k is None:
            ret = np.dot(X, self.w)
        else:
            ret = np.dot(X, self.w[k])

        return ret


    def get_distance_boundary(self, X, x_sensitive):

        """
            returns two vals

            distance_boundary_arr:
                arr with distance to boundary, each groups owns w is applied on it
            distance_boundary_dict:
                dict of the form s_attr_group (points from group 0/1) -> w_group (boundary of group 0/1) -> distances for this group with this boundary

        """

        distances_boundary_dict = {} # s_attr_group (0/1) -> w_group (0/1) -> distances


        if not isinstance(self.w, dict): # we have one model for the whole data
            distance_boundary_arr = self.decision_function(X)

            for attr in set(x_sensitive): # there is only one boundary, so the results with this_group and other_group boundaries are the same

                distances_boundary_dict[attr] = {}
                idx = x_sensitive == attr

                for k in set(x_sensitive):
                    distances_boundary_dict[attr][k] = self.decision_function(X[idx]) # apply same decision function for all the sensitive attrs because same w is trained for everyone


        else: # w is a dict
            distance_boundary_arr = np.zeros(X.shape[0])


            for attr in set(x_sensitive):

                distances_boundary_dict[attr] = {}
                idx = x_sensitive == attr
                X_g = X[idx]

                distance_boundary_arr[idx] = self.decision_function(X_g, attr) # each group gets decision with their own boundary

                for k in self.w.keys():
                    distances_boundary_dict[attr][k] = self.decision_function(X_g, k) # each group gets a decision with both boundaries

        return distance_boundary_arr, distances_boundary_dict


    def get_di_cons_single_boundary(self, X, y, x_sensitive, w, cov_thresh):

        """
        Parity impact constraint
        """

        assert(self.train_multiple == False) # di cons is just for a single boundary clf
        assert(cov_thresh >= 0) # covariance thresh has to be a small positive number

        constraints = []
        z_i_z_bar = x_sensitive - np.mean(x_sensitive)

        fx = X*w
        prod = sum_entries( mul_elemwise(z_i_z_bar, fx) ) / X.shape[0]


        constraints.append( prod <=  cov_thresh )
        constraints.append( prod >= -cov_thresh )

        return constraints


    def get_preferred_cons(self, X, x_sensitive, w, cons_type, s_val_to_cons_sum=None):

        """
            No need to pass s_val_to_cons_sum for preferred treatment (envy free) constraints

            For details on cons_type, see the documentation of fit() function
        """

        constraints = []

        if cons_type in [1,2,3]: # 1 - pref imp, 2 - EF, 3 - pref imp & EF

            prod_dict = {0:{}, 1:{}} # s_attr_group (0/1) -> w_group (0/1) -> val
            for val in set(x_sensitive):
                idx = x_sensitive == val
                X_g = X[idx]
                num_g = X_g.shape[0]

                for k in w.keys(): # get the distance with each group's w
                    prod_dict[val][k] = sum_entries(  max_elemwise(0, X_g*w[k])   ) / num_g



        else:
            raise Exception("Invalid constraint type")


        if cons_type == 1 or cons_type == 3: # 1 for preferred impact -- 3 for preferred impact and envy free

            constraints.append( prod_dict[0][0] >= s_val_to_cons_sum[0][0] )
            constraints.append( prod_dict[1][1] >= s_val_to_cons_sum[1][1] )



        if cons_type == 2 or cons_type == 3: # envy free
            constraints.append( prod_dict[0][0] >= prod_dict[0][1] )
            constraints.append( prod_dict[1][1] >= prod_dict[1][0] )

        return constraints

SyntaxError: ignored